# Imports

In [1]:
import geopandas as gpd
import pandas as pd
import fiona
import os
import matplotlib.pyplot as plt
import folium
from zipfile import ZipFile
import seaborn as sns
import numpy as np
import datetime as dt
from collections import Counter
import requests
from ipywidgets import interact, interactive, fixed, interact_manual, IntSlider
from shapely.geometry import LineString, Point
from shapely import wkt
from dotenv import load_dotenv, find_dotenv
from folium.plugins import MarkerCluster, HeatMap, BeautifyIcon
from folium.map import LayerControl, Layer, FeatureGroup
from folium.vector_layers import Circle, CircleMarker
import matplotlib
import ast

# Data

In [2]:
# import renatl info and drop irrelevant columns
bike_df_all = pd.read_csv('raw_data/bike_loans.csv', sep=';').drop(columns=['Importe', 
                                                                        'DescripcionImporte', 
                                                                        'operario'])
bike_df_all['Fecha_Prestamo'] = bike_df_all['Fecha_Prestamo'].astype('datetime64[ns]')
bike_df_all['Fecha_Devolucion'] = bike_df_all['Fecha_Devolucion'].astype('datetime64[ns]')

bike_df_all.dtypes

Id_Historico_Prestamo               int64
Id_Usuario                          int64
Id_Tag_Bicicleta                   object
Fecha_Prestamo             datetime64[ns]
Fecha_Devolucion           datetime64[ns]
Id_Aparcamiento_Origen              int64
Posicion_Origen                     int64
Id_Aparcamiento_Destino           float64
Posicion_Destino                  float64
Num_Bici_Hist                     float64
dtype: object

In [3]:
bike_df_all = bike_df_all.dropna()

# Locate individual bike at station

In [4]:
# filter to 2019 data => most representative year
bikes_19 = bike_df_all[bike_df_all.Fecha_Devolucion.dt.year == 2019].reset_index()

In [5]:
# assign df a individual hour of the year per row
def get_hr(x):
    time = x
    hour = dt.datetime(time.year, time.month,time.day,time.hour)
    return hour
    
bikes_19.loc[:,'hour'] = bikes_19.Fecha_Devolucion.apply(get_hr)

In [6]:
# make sure df is sorted chronologically
bikes_19 = bikes_19.sort_values(by='Fecha_Devolucion')

# list all the appearing bikes and stations per individual hour of the year
bikes_19_by_bike = bikes_19.copy().groupby('hour').agg({'Id_Tag_Bicicleta':lambda x: list(x), 'Id_Aparcamiento_Destino':lambda x: list(x)})

# only keep the _last_ entry of a bike per hour (dict will overwrite)
bikes_19_by_bike['bikes_loc'] = None
for c, row in bikes_19_by_bike.iterrows():
#     print(c) # check progress
    dct = {}
    for i, bike in enumerate(row.Id_Tag_Bicicleta):
        dct[bike] = row.Id_Aparcamiento_Destino[i]
    bikes_19_by_bike.at[c, 'bikes_loc'] = dct

# move hour to columns and only leave the dict
bikes_19_by_bike = bikes_19_by_bike[['bikes_loc']].reset_index()

# count the appearance of each station per hour
bikes_19_by_bike['station_count'] = None
for c, row in bikes_19_by_bike.iterrows():
    stations = row.bikes_loc.values()
    for station in stations:
        dct = dict(Counter(stations))
    bikes_19_by_bike.at[c, 'station_count'] = dct
        

# make dict column visible
bikes_19_by_bike.head(50).style.set_properties(subset=['bikes_loc'], **{'width': '150px'})

In [7]:
bikes_19_by_bike.to_csv('raw_data/stations_count_hr.csv')